In [11]:
import dt_apriltags as atag
import cv2
import numpy as np
import os

In [12]:
# create detector
detector = atag.Detector(families="tagStandard41h12")

In [13]:

calibration_images = [
    # "testimages/cal2.jpg",
    # "testimages/cal3.jpg",
    # "testimages/cal4.jpg",
    # "testimages/cal5.jpg",
    # "testimages/cal6.jpg",
    # "testimages/cal7.jpg",
    # "testimages/cal8.jpg",
    # "testimages/cal9.jpg",
    # "testimages/cal10.jpg",
    # "testimages/cal11.jpg",
    # "testimages/cal12.jpg",
    # "testimages/cal13.jpg",
    # "testimages/cal14.jpg",
    # "testimages/cal15.jpg",
    # "testimages/cal16.jpg",
    "rs_cal_big/"
]
for p in calibration_images:
    if os.path.isdir(p):
        calibration_images.remove(p)
        files = os.listdir(p)
        files = [p + f for f in files if os.path.isfile(p+'/'+f)]
        calibration_images.extend(files)

calibration_grid = (3, 2)
calibration_sep = (0.5, 2.9375)
calibration_size = 8


# calibration_grid = (9, 5)
# calibration_sep = (0.25, 0.5)
# calibration_size = 0.75

calibration_offset = (
    calibration_sep[0] + calibration_size,
    calibration_sep[1] + calibration_size,
)
pixel_per_tag = 9
pixel_width = calibration_size / pixel_per_tag
tag_count = calibration_grid[0] * calibration_grid[1]
tag_corners = [
    [2 * pixel_width, 2 * pixel_width],
    [2 * pixel_width, 7 * pixel_width],
    [7 * pixel_width, 7 * pixel_width],
    [7 * pixel_width, 2 * pixel_width],
]
print("Using files")
print(calibration_images)

Using files
['rs_cal_big/69.jpg', 'rs_cal_big/49.jpg', 'rs_cal_big/60.jpg', 'rs_cal_big/35.jpg', 'rs_cal_big/19.jpg', 'rs_cal_big/46.jpg', 'rs_cal_big/34.jpg', 'rs_cal_big/8.jpg', 'rs_cal_big/37.jpg', 'rs_cal_big/39.jpg', 'rs_cal_big/74.jpg', 'rs_cal_big/4.jpg', 'rs_cal_big/30.jpg', 'rs_cal_big/31.jpg', 'rs_cal_big/72.jpg', 'rs_cal_big/1.jpg', 'rs_cal_big/77.jpg', 'rs_cal_big/20.jpg', 'rs_cal_big/64.jpg', 'rs_cal_big/81.jpg', 'rs_cal_big/76.jpg', 'rs_cal_big/40.jpg', 'rs_cal_big/78.jpg', 'rs_cal_big/50.jpg', 'rs_cal_big/21.jpg', 'rs_cal_big/11.jpg', 'rs_cal_big/22.jpg', 'rs_cal_big/52.jpg', 'rs_cal_big/57.jpg', 'rs_cal_big/6.jpg', 'rs_cal_big/53.jpg', 'rs_cal_big/9.jpg', 'rs_cal_big/26.jpg', 'rs_cal_big/79.jpg', 'rs_cal_big/15.jpg', 'rs_cal_big/5.jpg', 'rs_cal_big/32.jpg', 'rs_cal_big/13.jpg', 'rs_cal_big/2.jpg', 'rs_cal_big/17.jpg', 'rs_cal_big/12.jpg', 'rs_cal_big/0.jpg', 'rs_cal_big/43.jpg', 'rs_cal_big/73.jpg', 'rs_cal_big/38.jpg', 'rs_cal_big/45.jpg', 'rs_cal_big/42.jpg', 'rs_cal_

In [14]:
truepoints = []
for x in range(calibration_grid[0]):
    for y in range(calibration_grid[1]):
        origin = (x * calibration_offset[0], y * calibration_offset[1])
        for corner in tag_corners:
            truepoints.append([origin[0] + corner[0], origin[1] + corner[1], 0])


xgrid = np.array([p[0] for p in truepoints])
ygrid = np.array([p[1] for p in truepoints])
zgrid = np.array([p[2] for p in truepoints])

opoints = np.dstack((xgrid, ygrid, zgrid)).reshape((-1, 1, 3)).astype(np.float32)

In [15]:
imagesize = None

truepoints = []
ipoints = []

usableImageCount = 0
unusable = []

for filename in calibration_images:
    rgb = cv2.imread(filename)

    if rgb is None:
        print("warning: error opening {}, skipping".format(filename))
        continue
    # elif imagesize is not None:
    #     print("loaded " + filename + " of size " + str(imagesize[0]) + ", " + str(imagesize[1]))


    cursize = (rgb.shape[1], rgb.shape[0])

    if imagesize is None:
        imagesize = cursize
    else:
        assert imagesize == cursize, str(imagesize) + " != " +str(cursize)


    if len(rgb.shape) == 3:
        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    else:
        gray = rgb

    results = detector.detect(gray)
    if len(results) == tag_count:
        usableImageCount += 1
        truepoints.append(opoints)
        foundpoints = []
        for tag in results:
            corners = tag.corners
            for c in corners:
                foundpoints.append([c])

        ipoints.append(foundpoints)
    else:
        # print("could not find all apriltags")
        unusable.append([str(len(results)) + "/" + str(tag_count), filename])


flags = (
    cv2.CALIB_ZERO_TANGENT_DIST
    | cv2.CALIB_FIX_K1
    | cv2.CALIB_FIX_K2
    | cv2.CALIB_FIX_K3
    | cv2.CALIB_FIX_K4
    | cv2.CALIB_FIX_K5
    | cv2.CALIB_FIX_K6
)
truepoints = np.array(truepoints, dtype=np.float32)
# truepoints = truepoints.reshape((-1,1,3)).astype(np.float32)

ipoints = np.array(ipoints, dtype=np.float32)
# ipoints = ipoints.reshape((-1,1,3)).astype(np.float32)

# print(truepoints.shape)
# print(ipoints.shape)

# uncomment to delete unusable images
# for el in unusable:
#     os.remove(el[1])

for el in unusable:
    print(el)
    
print(
    f"Used {usableImageCount}/{len(calibration_images)} images.  See above for list of unusable images."
)

retval, K, dcoeffs, rvecs, tvecs = cv2.calibrateCamera(
    truepoints,
    ipoints,
    imagesize,
    cameraMatrix=None,
    distCoeffs=np.zeros(5),
    flags=flags,
)
fx = K[0, 0]
fy = K[1, 1]
cx = K[0, 2]
cy = K[1, 2]

params = (fx, fy, cx, cy)

print()
print("all units below measured in pixels:")
print("  fx = {}".format(K[0, 0]))
print("  fy = {}".format(K[1, 1]))
print("  cx = {}".format(K[0, 2]))
print("  cy = {}".format(K[1, 2]))
print()
print("pastable into Python:")
print("  fx, fy, cx, cy = {}".format(repr(params)))
print()

['3/6', 'rs_cal_big/49.jpg']
['4/6', 'rs_cal_big/60.jpg']
['2/6', 'rs_cal_big/35.jpg']
['4/6', 'rs_cal_big/46.jpg']
['4/6', 'rs_cal_big/37.jpg']
['1/6', 'rs_cal_big/39.jpg']
['2/6', 'rs_cal_big/30.jpg']
['0/6', 'rs_cal_big/31.jpg']
['2/6', 'rs_cal_big/77.jpg']
['4/6', 'rs_cal_big/64.jpg']
['3/6', 'rs_cal_big/81.jpg']
['3/6', 'rs_cal_big/76.jpg']
['3/6', 'rs_cal_big/50.jpg']
['3/6', 'rs_cal_big/57.jpg']
['4/6', 'rs_cal_big/5.jpg']
['0/6', 'rs_cal_big/32.jpg']
['3/6', 'rs_cal_big/0.jpg']
['4/6', 'rs_cal_big/73.jpg']
['3/6', 'rs_cal_big/38.jpg']
['4/6', 'rs_cal_big/45.jpg']
['2/6', 'rs_cal_big/42.jpg']
['4/6', 'rs_cal_big/47.jpg']
['0/6', 'rs_cal_big/29.jpg']
['5/6', 'rs_cal_big/80.jpg']
['5/6', 'rs_cal_big/25.jpg']
['0/6', 'rs_cal_big/70.jpg']
['5/6', 'rs_cal_big/65.jpg']
['4/6', 'rs_cal_big/59.jpg']
['4/6', 'rs_cal_big/63.jpg']
['3/6', 'rs_cal_big/33.jpg']
['0/6', 'rs_cal_big/56.jpg']
['5/6', 'rs_cal_big/48.jpg']
['3/6', 'rs_cal_big/27.jpg']
['0/6', 'rs_cal_big/62.jpg']
['5/6', 'rs_cal_

Used 32/65 images.  See above for list of unusable images.

all units below measured in pixels:
  fx = 0.2990620031152041
  fy = 480.06231408700006
  cx = 1125.0716068477543
  cy = 525.6012496992596

pastable into Python:
  fx, fy, cx, cy = (0.2990620031152041, 480.06231408700006, 1125.0716068477543, 525.6012496992596)



Used 73/117 images.  See above for list of unusable images.

all units below measured in pixels:
  fx = 3.891925071839006
  fy = 135.79439965557637
  cx = 1064.347741959847
  cy = 501.45362561786624

pastable into Python:
  fx, fy, cx, cy = (3.891925071839006, 135.79439965557637, 1064.347741959847, 501.45362561786624)

  Used 4/4 images.  See above for list of unusable images.

all units below measured in pixels:
  fx = -0.0264359920352824
  fy = 490.89523976369776
  cx = 1022.9325623173796
  cy = 575.0560732266949

pastable into Python:
  fx, fy, cx, cy = (-0.0264359920352824, 490.89523976369776, 1022.9325623173796, 575.0560732266949)



Used 43/82 images.  See above for list of unusable images.

all units below measured in pixels:
  fx = 422.81293768009186
  fy = 1.4992536896751996
  cx = 1036.139416213706
  cy = 558.6924006691295

pastable into Python:
  fx, fy, cx, cy = (422.81293768009186, 1.4992536896751996, 1036.139416213706, 558.6924006691295)
